In [1]:
import struct
import uuid
import binascii
import enum
import datetime as dt
import numpy as np
import queue

In [2]:
packet = bytes.fromhex('')

In [3]:
class binaryPacket:
    def __init__(self, payload:bytes, packetClass:int, packetID:int, sourceUUID:uuid.UUID, destUUID:uuid.UUID):
        self._payload = payload
        self._class = packetClass
        self._id = packetID
        self._source = sourceUUID
        self._dest = destUUID
    
    def to_bytes(self):
        payloadLen = len(self._payload)
        header = struct.pack("<BB", 0xE4, 0xEB) + \
                             self._source.bytes + \
                             self._dest.bytes + \
                             struct.pack("<BBH", self._class, self._id, payloadLen)
        pktCksum = binascii.crc_hqx(header, 0xFFFF).to_bytes(2, "big")
        msg = header + pktCksum + self._payload
        cksum = binascii.crc_hqx(msg, 0xFFFF).to_bytes(2, "big")
        return msg + cksum
    
    def getClassIDCode(self):
        return (self._class << 8) | self._id
    
    def __str__(self):
        string = self.to_bytes().hex().upper()
        length = 4
        return '%s' % ' '.join(string[i:i + length] for i in range(0, len(string), length))
    
    def __repr__(self):
        return str(self)
    
    def __eq__(self, packet):
        return self.to_bytes() == packet.to_bytes()
    
    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        return binaryPacket(payload, pcls, pid, uuid.UUID(bytes=srcUUID), uuid.UUID(bytes=destUUID))
    
    @classmethod
    def parseHeader(cls, packet:bytes):
        if binascii.crc_hqx(packet, 0xFFFF) != 0:
            raise RuntimeError("Checksum verification failed")
            
        if binascii.crc_hqx(packet[0:0x0028], 0xFFFF) != 0:
            raise RuntimeError("Header checksum verification failed")
        
        if len(packet) < 0x2A:
            raise RuntimeError("Packet too short!")
            
            
        s1, s2, pcls, pid, _ = struct.unpack("<BB16x16xBBH", packet[0:0x26])
        srcUUID = bytes(packet[0x0002:0x0012])
        destUUID = bytes(packet[0x0012:0x0022])
        if s1 != 0xE4 or s2 != 0xEB:
            raise RuntimeError("Not a packet!")
        payload = packet[0x28:-2]
        return srcUUID, destUUID, pcls, pid, payload
    
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return True

In [4]:
class E4E_Data_IMU(binaryPacket):
    __class = 0x04
    __id = 0x00
    __VERSION = 0x01

    def __init__(self, src:uuid.UUID, dest:uuid.UUID, accX, accY, accZ, gyroX, gyroY, gyroZ, magX, magY, magZ, timestamp:dt.datetime = None):
        self.acc = [accX, accY, accZ]
        self.gyro = [gyroX, gyroY, gyroZ]
        self.mag = [magX, magY, magZ]
        if timestamp is None:
            timestamp = dt.datetime.now()
        self.timestamp = timestamp
        
        
        payload = struct.pack("<BBQ3f3f3f",
                                   self.__VERSION,
                                   0x00,
                                   int(timestamp.timestamp() * 1e3),
                                   *self.acc,
                                   *self.gyro,
                                   *self.mag)
        super(E4E_Data_IMU, self).__init__(payload, self.__class, self.__id, src, dest)
        
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return packetClass == self.__class and packetID == self.__id
    
    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        ver, _, timestamp_ms, accX, accY, accZ, gyroX, gyroY, gyroZ, magX, magY, magZ = struct.unpack("<BBQ3f3f3f", payload)
        if ver != cls.__VERSION:
            return RuntimeError("Unknown packet version!")
        
        timestamp = dt.datetime.fromtimestamp(timestamp_ms / 1e3)
        return E4E_Data_IMU(src=uuid.UUID(bytes=srcUUID),
                           dest=uuid.UUID(bytes=destUUID),
                           accX = accX,
                           accY = accY,
                           accZ = accZ,
                           gyroX = gyroX,
                           gyroY = gyroY,
                           gyroZ = gyroZ,
                           magX = magX,
                           magY = magY,
                           magZ = magZ,
                           timestamp=timestamp)

In [5]:
class E4E_Data_Audio_raw8(binaryPacket):
    __class = 0x04
    __id = 0x01
    __VERSION = 0x01
    
    def __init__(self, audioData:np.ndarray, src:uuid.UUID, dest:uuid.UUID, timestamp:dt.datetime=None):
        assert(len(audioData.shape) == 2)
        nChannels = audioData.shape[0]
        nSamples = audioData.shape[1]
        if timestamp is None:
            timestamp = dt.datetime.now()
        
        self.audioData = audioData
        payload = struct.pack("<BBHQ",
                             self.__VERSION,
                             nChannels,
                              nSamples,
                              int(timestamp.timestamp() * 1e3))
        for channel in range(nChannels):
            for sampleIdx in range(nSamples):
                payload += struct.pack("<B", int(audioData[channel, sampleIdx]))
        super(E4E_Data_Audio_raw8, self).__init__(payload, self.__class, self.__id, src, dest)
    
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return packetClass == self.__class and packetID == self.__id
    
    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        ver, nChannels, nSamples, timestamp_ms = struct.unpack("<BBHQ", payload[0:0x0C])
        audioBytes = payload[0x0C:]
        assert(len(audioBytes) == nChannels * nSamples)
        idx = 0
        audioData = np.zeros((nChannels, nSamples))
        for channel in range(nChannels):
            for sampleIdx in range(nSamples):
                audioData[channel, sampleIdx] = audioBytes[idx]
                idx += 1
        timestamp = dt.datetime.fromtimestamp(timestamp_ms / 1e3)
        return E4E_Data_Audio_raw8(audioData, uuid.UUID(bytes=srcUUID), uuid.UUID(bytes=destUUID), timestamp)

In [6]:
class E4E_Data_Audio_raw16(binaryPacket):
    __class = 0x04
    __id = 0x02
    __VERSION = 0x01
    
    def __init__(self, audioData:np.ndarray, src:uuid.UUID, dest:uuid.UUID, timestamp:dt.datetime=None):
        assert(len(audioData.shape) == 2)
        nChannels = audioData.shape[0]
        nSamples = audioData.shape[1]
        if timestamp is None:
            timestamp = dt.datetime.now()
        
        self.audioData = audioData
        payload = struct.pack("<BBHQ",
                             self.__VERSION,
                             nChannels,
                              nSamples,
                              int(timestamp.timestamp() * 1e3))
        for channel in range(nChannels):
            for sampleIdx in range(nSamples):
                payload += struct.pack("<H", int(audioData[channel, sampleIdx]))
        super(E4E_Data_Audio_raw16, self).__init__(payload, self.__class, self.__id, src, dest)
    
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return packetClass == self.__class and packetID == self.__id
    
    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        ver, nChannels, nSamples, timestamp_ms = struct.unpack("<BBHQ", payload[0:0x0C])
        audioBytes = payload[0x0C:]
        assert(len(audioBytes) == nChannels * nSamples * 2)
        idx = 0
        audioData = np.zeros((nChannels, nSamples))
        for channel in range(nChannels):
            for sampleIdx in range(nSamples):
                audioData[channel, sampleIdx], = struct.unpack("<H", audioBytes[idx * 2 : idx * 2 + 2])
                idx += 1
        timestamp = dt.datetime.fromtimestamp(timestamp_ms / 1e3)
        return E4E_Data_Audio_raw16(audioData, uuid.UUID(bytes=srcUUID), uuid.UUID(bytes=destUUID), timestamp)

In [7]:
class E4E_Data_Raw_File_Header(binaryPacket):
    __class = 0x04
    __id = 0xFC
    __VERSION = 0x01
    
    def __init__(self, fileID:int, filename:str, MIMEType:str, fileSize:int, fileTime:dt.datetime, src:uuid.UUID, dest:uuid.UUID):
        self.fileID = fileID
        self.filename = filename
        self.mimeType = MIMEType
        self.fileSize = fileSize
        self.fileTime = fileTime
        payload = struct.pack("<BBHHQQ",
                             self.__VERSION,
                             fileID,
                             len(filename),
                             len(MIMEType),
                             fileSize,
                             int(fileTime.timestamp() * 1e3))
        payload += filename.encode('ascii')
        payload += MIMEType.encode('ascii')
        super(E4E_Data_Raw_File_Header, self).__init__(payload, self.__class, self.__id, src, dest)
        
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return packetClass == self.__class and packetID == self.__id

    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        ver, fileID, filenameLen, mimeTypeLen, fileSize, fileTimestamp = struct.unpack("<BBHHQQ", payload[0:0x16])
        filename = payload[0x16:0x16 + filenameLen].decode()
        mimeType = payload[0x16 + filenameLen:].decode()
        timestamp = dt.datetime.fromtimestamp(fileTimestamp / 1e3)
        return E4E_Data_Raw_File_Header(fileID, filename, mimeType, fileSize, timestamp, uuid.UUID(bytes=srcUUID), uuid.UUID(bytes=destUUID))

In [8]:
class E4E_Data_Raw_File_CTS(binaryPacket):
    __class = 0x04
    __id = 0xFD
    __VERSION = 0x01
    
    def __init__(self, fileID:int, ack:bool, src:uuid.UUID, dest:uuid.UUID):
        self.fileID = fileID
        self.ack = ack
        payload = struct.pack("<BBB", self.__VERSION, fileID, int(ack))
        super(E4E_Data_Raw_File_CTS, self).__init__(payload, self.__class, self.__id, src, dest)
        
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return packetClass == self.__class and packetID == self.__id

    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        fileID = payload[1]
        if payload[2] == 1:
            ack = True
        else:
            ack = False
        return E4E_Data_Raw_File_CTS(fileID, ack, uuid.UUID(bytes=srcUUID), uuid.UUID(bytes=destUUID))

In [9]:
class E4E_Data_Raw_File_ACK(binaryPacket):
    __class = 0x04
    __id = 0xFE
    __VERSION = 0x01
    
    def __init__(self, fileID:int, seq:int, ack:bool, src:uuid.UUID, dest:uuid.UUID):
        self.fileID = fileID
        self.seq = seq
        self.ack = ack
        payload = struct.pack("<BBHB", self.__VERSION, fileID, seq, int(ack))
        super(E4E_Data_Raw_File_ACK, self).__init__(payload, self.__class, self.__id, src, dest)
        
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return packetClass == self.__class and packetID == self.__id

    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        ver, fileID, seq, ackInt = struct.unpack("<BBHB", payload)
        if ackInt == 1:
            ack = True
        else:
            ack = False
        return E4E_Data_Raw_File_ACK(fileID, seq, ack, uuid.UUID(bytes=srcUUID), uuid.UUID(bytes=destUUID))

In [10]:
class E4E_Data_Raw_File(binaryPacket):
    __class = 0x04
    __id = 0xFF
    __VERSION = 0x01
    
    def __init__(self, fileID:int, seq:int, blob:bytes, src:uuid.UUID, dest:uuid.UUID):
        self.fileID = fileID
        self.seq = seq
        self.blob = blob
        payload = struct.pack("<BBHQ", self.__VERSION, fileID, seq, len(blob))
        payload += blob
        super(E4E_Data_Raw_File, self).__init__(payload, self.__class, self.__id, src, dest)
        
    @classmethod
    def matches(cls, packetClass:int, packetID:int):
        return packetClass == self.__class and packetID == self.__id

    @classmethod
    def from_bytes(cls, packet:bytes):
        srcUUID, destUUID, pcls, pid, payload = cls.parseHeader(packet)
        ver, fileID, seq, blobLen = struct.unpack("<BBHQ", payload[0:0x0C])
        blob = payload[0x0C:0x0C + blobLen]
        return E4E_Data_Raw_File(fileID, seq, blob, uuid.UUID(bytes=srcUUID), uuid.UUID(bytes=destUUID))

In [11]:
class binaryPacketParser:
    class State(enum.Enum):
        FIND_SYNC1 = 0
        FIND_SYNC2 = 1
        HEADER = 2
        HEADER_CKSUM = 3
        HEADER_VALIDATE = 4
        PAYLOAD = 5
        CKSUM = 6
        VALIDATE = 7
        RECYCLE = 8
    packetMap = {
        0x0400: E4E_Data_IMU,
        0x0401: E4E_Data_Audio_raw8,
        0x0402: E4E_Data_Audio_raw16,
        0x04FC: E4E_Data_Raw_File_Header,
        0x04FD: E4E_Data_Raw_File_CTS,
        0x04FE: E4E_Data_Raw_File_ACK,
        0x04FF: E4E_Data_Raw_File
    }
    
    HEADER_LEN = 0x0026
    
    def __init__(self):
        self.__state = self.State.FIND_SYNC1
        self.__payloadLen = 0
        self.__buffer = None
        self.__data = queue.Queue()
        
    def parseByte(self, data:int):
        self.__data.put(data)
        while not self.__data.empty():
            retval = self._parseByte()
        return retval
            
    def _parseByte(self):
        data = self.__data.get_nowait()
#         print("%s: 0x%02x" % (self.__state, data))

        if self.__state is self.State.FIND_SYNC1:
            if data == 0xE4:
                self.__state = self.State.FIND_SYNC2
                self.__buffer = bytearray()
                self.__buffer.append(data)
            return None
        elif self.__state is self.State.FIND_SYNC2:
            if data == 0xEB:
                self.__state = self.State.HEADER
                self.__buffer.append(data)
            else:
                self.__state = self.State.FIND_SYNC1
            return None
        elif self.__state is self.State.HEADER:
            self.__buffer.append(data)
            if len(self.__buffer) == self.HEADER_LEN:
                self.__state = self.State.HEADER_CKSUM
                self.__payloadLen, = struct.unpack('<H', self.__buffer[self.HEADER_LEN - 2:self.HEADER_LEN])
            return None
        elif self.__state is self.State.HEADER_CKSUM:
            self.__buffer.append(data)
            self.__state = self.State.HEADER_VALIDATE
        elif self.__state is self.State.HEADER_VALIDATE:
            self.__buffer.append(data)
            if binascii.crc_hqx(self.__buffer, 0xFFFF) != 0:
#                 print(self.__buffer.hex())
#                 print(binascii.crc_hqx(self.__buffer, 0xFFFF).to_bytes(2, "big").hex())
#                 print("Invalid header")
                # Recycle everything
                self.__state = self.State.FIND_SYNC1
                self.__recycleBuffer = self.__buffer[2:]
                while not self.__data.empty():
                    self.__recycleBuffer.append(self.__data.get_nowait())
                for byte in self.__recycleBuffer:
                    self.__data.put(byte)
            else:
                self.__state = self.State.PAYLOAD
            return None
        elif self.__state is self.State.PAYLOAD:
            self.__buffer.append(data)
            if len(self.__buffer) == self.__payloadLen + self.HEADER_LEN + 2:
                self.__state = self.State.CKSUM
            return None
        elif self.__state is self.State.CKSUM:
            self.__buffer.append(data)
            self.__state = self.State.VALIDATE
            return None
        elif self.__state is self.State.VALIDATE:
            self.__buffer.append(data)
            if binascii.crc_hqx(self.__buffer, 0xFFFF) != 0:
#                 print(self.__buffer.hex())
#                 print(binascii.crc_hqx(self.__buffer, 0xFFFF).to_bytes(2, "big").hex())
                raise RuntimeError("Checksum verification failed")
            packetID, = struct.unpack('>H', self.__buffer[0x0022:0x0024])
            self.__state = self.State.FIND_SYNC1
            if packetID not in self.packetMap:
                return binaryPacket.from_bytes(self.__buffer)
            else:
                return self.packetMap[packetID].from_bytes(self.__buffer)
    
    def parseBytes(self, data:bytes):
        packets = []
        for byte in data:
            self.__data.put(byte)
        while not self.__data.empty():
            retval = self._parseByte()
            if retval is not None:
                packets.append(retval)
        return packets

In [12]:
imuPacket = E4E_Data_IMU(src=uuid.uuid4(),
                        dest=uuid.uuid4(),
                        accX=1.0,
                        accY=2.0,
                        accZ=3.0,
                        gyroX=4.0,
                        gyroY=5.0,
                        gyroZ=6.0,
                        magX=7.0,
                        magY=8.0,
                        magZ=9.0,
                        timestamp=dt.datetime(2020, 12, 24, 0, 13, 13))
print(imuPacket)
print(hex(len(imuPacket.to_bytes())))
parser = binaryPacketParser()
packets = parser.parseBytes(imuPacket.to_bytes())
print(packets)
print(type(packets[0]))

E4EB 7551 EB2C 6564 45E1 883E E1E4 FB23 A90C B7C1 79B5 0EF1 4259 B24C 1B01 8EB8 79A9 0400 2E00 BD2A 0100 A81D CF93 7601 0000 0000 803F 0000 0040 0000 4040 0000 8040 0000 A040 0000 C040 0000 E040 0000 0041 0000 1041 BA7B
0x58
[E4EB 7551 EB2C 6564 45E1 883E E1E4 FB23 A90C B7C1 79B5 0EF1 4259 B24C 1B01 8EB8 79A9 0400 2E00 BD2A 0100 A81D CF93 7601 0000 0000 803F 0000 0040 0000 4040 0000 8040 0000 A040 0000 C040 0000 E040 0000 0041 0000 1041 BA7B]
<class '__main__.E4E_Data_IMU'>


In [13]:
rng = np.random.default_rng()
audioData = rng.integers(255, size=(2, 256))
audioPacket = E4E_Data_Audio_raw8(audioData = audioData,
                                 src=uuid.uuid4(),
                                 dest=uuid.uuid4(),
                                 timestamp=dt.datetime(2020, 12, 24, 20, 12, 8))
print(audioPacket)
print(hex(len(audioPacket.to_bytes())))
packets = parser.parseBytes(audioPacket.to_bytes())
print(packets)
print(type(packets[0]))

E4EB 114D D51F 40AD 4EB1 BB8E 0F5D 91AA 299D BA9E 9A9C 4BA6 455B 8E1E 173E 068C 0F70 0401 0C02 C06F 0102 0001 C0C1 1898 7601 0000 82A8 3240 0E85 9D58 51F1 6F9B 242B 88E4 A3C1 368E 7B62 25C7 5A72 9B2A 4273 029E F927 CEFC EBD7 D21A 7E76 5CA0 8379 3BE9 0404 40D6 9FEA 30FD D1AB B54C 66EC 391B C959 6E8B 62CD 6DE0 5979 CF45 B56B A849 033F 364E E601 5748 72F0 B399 B9F6 DFB7 E9AF 7DCA 0D90 E3F7 BF8A 9F38 B9F7 AB9E 10CC 4ECD 6A1C 10E8 867E EEE4 F5BD 12B5 66FD 4C1B F979 33CC 7B1E 6978 E85C 4DC1 A3D4 F9BA 5474 9108 1783 1D02 FA35 BC56 C7AA 4E8A 84F3 7AE7 7D37 C9A6 9E9C 683D 0113 F7EB 1B47 5DEC FA0C C78E BA8A 4895 5F59 562B 2147 BEB2 5371 8B67 E193 5C6A 1553 CE5A 9B49 C27E 40AF 204D 23D7 6EC1 ECEA 22E0 CDD6 0DB4 F498 8442 60CB 50D4 2A35 ED95 29C0 DE0E 6B05 7BA1 0214 7E20 5298 FD87 8A13 E3D2 0066 49F8 F202 98DB A4FC 7942 1C12 7B1B 99E1 3F3F 3233 58D4 FB0F 2DA5 E619 6569 88B0 C84B AD45 C98F 0733 FC5C 1E31 8BA9 34E1 B772 0288 834D 30CE F5B0 D6B9 6F1B 1DD7 40BC 9485 0A53 9544 2887 2C3D 8B32 86C7 766F 

In [14]:
rng = np.random.default_rng()
audioData = rng.integers(32767, size=(2, 256))
audioPacket = E4E_Data_Audio_raw16(audioData = audioData,
                                 src=uuid.uuid4(),
                                 dest=uuid.uuid4(),
                                 timestamp=dt.datetime(2020, 12, 24, 20, 12, 8))
print(audioPacket)
print(hex(len(audioPacket.to_bytes())))
packets = parser.parseBytes(audioPacket.to_bytes())
print(packets)
print(type(packets[0]))

E4EB 9127 972E 49D0 4149 8047 0FF6 FACB 05ED 15E2 5F79 8849 4941 B56B F473 6F54 2062 0402 0C04 3013 0102 0001 C0C1 1898 7601 0000 A043 1431 8636 5267 EE0A 6D07 4A2B 961F B10B 504C 2E7E 9E63 614A 4E2D C17F BE0C 2616 7A64 7F32 5D29 DF4C 0842 096D 3B31 546B DE6A A61E EA2D 4E06 883F 9522 7C6A E507 D054 2500 2010 4B30 505D CC10 8B7E 480B 2D00 0524 3B5B 6D36 7539 F80B A26E DE74 3604 4117 9349 7024 0729 A32F 8B7B 4A4C C957 6C47 F47C CE26 6712 997B 9232 C539 3A65 EF1A AC38 8822 6413 4E3C 4829 DB5F F705 683B F502 7834 4E4B B962 9B6A 6B44 133E 0460 9A45 715E 604F 345E A35B D415 C323 6F2D B276 AA5A B55D 4829 9B48 6204 0528 901E 4851 8F69 521B 8D13 7162 987C 9D3B F64B 146B 5468 0700 523A 1A5D AB2D F900 0926 A536 A25F 6964 8D3A 387A 926A D740 E95C E026 835A 483B CC0E 3615 6466 D857 9B1D 6838 B73E 7277 BF32 687F 044D CF41 1553 8A48 3728 405A 4B78 9913 A806 9766 7770 D73F 8C74 7739 7304 1E7C 2851 312A 8743 4D09 201A 101D BC3C 9D70 D848 1E4E 7627 4A3A C01E 3D31 1457 A726 123B F24A 704D 6D34 F62C 210E 

In [15]:
import os
rng = np.random.default_rng()

fileID = 0
fileName = "Binary Data Protocol Specification.ipynb"
mimeType = "application/x-ipynb+json"
src = uuid.uuid4()
dest = uuid.uuid4()

statResult = os.stat(fileName)
fileSize = statResult.st_size
fileTime = dt.datetime.fromtimestamp(statResult.st_ctime)

packet = E4E_Data_Raw_File_Header(fileID, fileName, mimeType, fileSize, fileTime, src, dest)
print(packet)
print(hex(len(packet.to_bytes())))
packets = parser.parseBytes(packet.to_bytes())
print(packets)
print(type(packets[0]))

E4EB 82A5 CCB8 2B0F 4C54 BE41 929E 79BE 735D 26BB D1D8 2E1C 4A21 90FA 34D1 5E21 B194 04FC 5600 B2A9 0100 2800 1800 DB43 0000 0000 0000 E2CF 4BFF 7601 0000 4269 6E61 7279 2044 6174 6120 5072 6F74 6F63 6F6C 2053 7065 6369 6669 6361 7469 6F6E 2E69 7079 6E62 6170 706C 6963 6174 696F 6E2F 782D 6970 796E 622B 6A73 6F6E 0BE8
0x80
[E4EB 82A5 CCB8 2B0F 4C54 BE41 929E 79BE 735D 26BB D1D8 2E1C 4A21 90FA 34D1 5E21 B194 04FC 5600 B2A9 0100 2800 1800 DB43 0000 0000 0000 E2CF 4BFF 7601 0000 4269 6E61 7279 2044 6174 6120 5072 6F74 6F63 6F6C 2053 7065 6369 6669 6361 7469 6F6E 2E69 7079 6E62 6170 706C 6963 6174 696F 6E2F 782D 6970 796E 622B 6A73 6F6E 0BE8]
<class '__main__.E4E_Data_Raw_File_Header'>


In [16]:
rng = np.random.default_rng()
src = uuid.uuid4()
dest = uuid.uuid4()

fileID = 0
ack = True

packet = E4E_Data_Raw_File_CTS(fileID, ack, src, dest)
print(packet)
print(hex(len(packet.to_bytes())))
packets = parser.parseBytes(packet.to_bytes())
print(packets)
print(type(packets[0]))

E4EB D5D0 E9DB 6107 438C BDD6 C518 A0E3 C903 BB68 D98F 9793 4958 8F4B CE84 5E04 3BB8 04FD 0300 6485 0100 0127 11
0x2d
[E4EB D5D0 E9DB 6107 438C BDD6 C518 A0E3 C903 BB68 D98F 9793 4958 8F4B CE84 5E04 3BB8 04FD 0300 6485 0100 0127 11]
<class '__main__.E4E_Data_Raw_File_CTS'>


In [17]:
rng = np.random.default_rng()
src = uuid.uuid4()
dest = uuid.uuid4()

fileID = 0
seq = 1
ack = False

packet = E4E_Data_Raw_File_ACK(fileID, seq, ack, src, dest)
print(packet)
print(hex(len(packet.to_bytes())))
packets = parser.parseBytes(packet.to_bytes())
print(packets)
print(type(packets[0]))

E4EB B085 C41E D74A 4E73 AE89 49F7 6384 7914 5F04 506B 663A 4854 AEF4 801F 6602 38D0 04FE 0500 B020 0100 0100 009D 61
0x2f
[E4EB B085 C41E D74A 4E73 AE89 49F7 6384 7914 5F04 506B 663A 4854 AEF4 801F 6602 38D0 04FE 0500 B020 0100 0100 009D 61]
<class '__main__.E4E_Data_Raw_File_ACK'>


In [18]:
rng = np.random.default_rng()
src = uuid.uuid4()
dest = uuid.uuid4()

fileID = 0
fileName = "Binary Data Protocol Specification.ipynb"
seq = 1
# Let us use a standard block size of 1024
with open(fileName, 'rb') as file:
    # Discard 0th block
    file.read(1024)
    # Keep 1st block
    blob = file.read(1024)
packet = E4E_Data_Raw_File(fileID, seq, blob, src, dest)
print(packet)
print(hex(len(packet.to_bytes())))
packets = parser.parseBytes(packet.to_bytes())
print(packets)
print(type(packets[0]))

E4EB F095 8A54 3A73 48BA 8151 7B96 DDF4 E7F9 0D7D 44F6 6933 47B2 AEB4 8456 C003 78E2 04FF 0C04 994D 0100 0100 0004 0000 0000 0000 2020 2020 7C22 0A20 2020 5D0A 2020 7D2C 0A20 207B 0A20 2020 2263 656C 6C5F 7479 7065 223A 2022 6D61 726B 646F 776E 222C 0A20 2020 226D 6574 6164 6174 6122 3A20 7B7D 2C0A 2020 2022 736F 7572 6365 223A 205B 0A20 2020 2022 2320 4865 6164 6572 2043 6865 636B 7375 6D5C 6E22 2C0A 2020 2020 2254 6865 2063 6865 636B 7375 6D20 7368 616C 6C20 6265 2063 616C 6375 6C61 7465 6420 6F76 6572 2074 6865 2065 6E74 6972 6574 7920 6F66 2074 6865 2070 6163 6B65 7420 6865 6164 6572 2028 6279 7465 7320 3078 3030 3030 2074 6F20 3078 3030 3235 2920 7573 696E 6720 7468 6520 4352 432D 3136 2063 6865 636B 7375 6D20 2870 6F6C 796E 6F6D 6961 6C20 3078 3130 3231 2C20 7374 6172 7469 6E67 2076 616C 7565 2030 7846 4646 462C 206E 6F20 7265 666C 6563 7469 6F6E 292E 2020 5468 6520 6368 6563 6B73 756D 2069 7473 656C 6620 7368 616C 6C20 6265 2073 746F 7265 6420 6173 2061 2031 3620 6269 7420 6269 

In [19]:
import math
import threading
import filecmp

rng = np.random.default_rng()
txUUID = uuid.uuid4()
rxUUID = uuid.uuid4()
txQ = queue.Queue()
rxQ = queue.Queue()

outputFileName = "test.dat"

if os.path.isfile(outputFileName):
    os.remove(outputFileName)

fileName = "Binary Data Protocol Specification.ipynb"
mimeType = "application/x-ipynb+json"

class FileSenderState(enum.Enum):
    Header=1
    WaitForCTS=2
    Sending=3
    GetAcks=4
    
class FileReceiverState(enum.Enum):
    WaitForHeader=1
    GetData=2
    GetUnsent=3

def sender(fname, mimeType, blockSize=1024, fileID=0):
    state = FileSenderState.Header
    parser = binaryPacketParser()

    statResult = os.stat(fname)
    fileSize = statResult.st_size
    fileTime = dt.datetime.fromtimestamp(statResult.st_ctime)
    
    nPackets = math.ceil(fileSize / blockSize)
    sent = [False] * nPackets
    acked = [False] * nPackets
   
    with open(fname, 'rb') as fileData:
        while True:
            if state == FileSenderState.Header:

                packet = E4E_Data_Raw_File_Header(fileID, fname, mimeType, nPackets, fileTime, txUUID, rxUUID)
                rxQ.put(packet.to_bytes())
                state = FileSenderState.WaitForCTS
            elif state == FileSenderState.WaitForCTS:
                data = txQ.get()
                packets = parser.parseBytes(data)
                if len(packets):
                    if isinstance(packets[0], E4E_Data_Raw_File_CTS):
                        state = FileSenderState.Sending
                    else:
                        raise RuntimeError("Tx Didn't get CTS")
                else:
                    raise RuntimeError("Tx didn't get a packet")
            elif state == FileSenderState.Sending:
                fileData.seek(0)
                for seq in range(nPackets):
                    blob = fileData.read(blockSize)
                    if not acked[seq]:
                        packet = E4E_Data_Raw_File(fileID, seq, blob, txUUID, rxUUID)
                        rxQ.put(packet.to_bytes())
                    sent[seq] = True
                state = FileSenderState.GetAcks
            elif state == FileSenderState.GetAcks:
                data = txQ.get()
                packets = parser.parseBytes(data)
                if len(packets):
                    if isinstance(packets[0], E4E_Data_Raw_File_ACK):
                        packet = packets[0]
                        if packet.fileID == fileID:
                            acked[packet.seq] = packet.ack
                        if packet.ack == False:
                            state == FileSenderState.Sending
                if all(acked):
                    return
                        
def receiver(fname):
    state = FileReceiverState.WaitForHeader
    parser = binaryPacketParser()
    fileID = None
    nPackets = None
    received = []
    fileData = {}
    while True:
        if state == FileReceiverState.WaitForHeader:
            data = rxQ.get()
            packets = parser.parseBytes(data)
            if len(packets):
                if isinstance(packets[0], E4E_Data_Raw_File_Header):
                    packet = packets[0]
                    # generally here, we would write to the filename indicated by the packet, but for
                    # this example, I don't want to destroy existing files
                    fileID = packet.fileID
                    nPackets = packet.fileSize
                    fileTime = packet.fileTime
                    response = E4E_Data_Raw_File_CTS(fileID, True, rxUUID, txUUID)
                    txQ.put(response.to_bytes())
                    state = FileReceiverState.GetData
                    received = [False] * nPackets
        elif state == FileReceiverState.GetData:
            data = rxQ.get()
            packets = parser.parseBytes(data)
            for packet in packets:
                assert(isinstance(packet, E4E_Data_Raw_File))
                assert(packet.fileID == fileID)
                received[packet.seq] = True
                fileData[packet.seq] = packet.blob
                response = E4E_Data_Raw_File_ACK(fileID, packet.seq, True, rxUUID, txUUID)
                txQ.put(response.to_bytes())
            if all(received):
                with open(fname, 'wb') as file:
                    for i in range(nPackets):
                        file.write(fileData[i])
                return

transmitThread = threading.Thread(target=sender, args=(fileName, mimeType))
receiveThread = threading.Thread(target=receiver, args=(outputFileName,))

transmitThread.start()
receiveThread.start()

transmitThread.join()
receiveThread.join()

filecmp.cmp(fileName, outputFileName)

True